# Rotating IP address with Bright Data and Python library requests

When scraping data from websites, regularly changing the IP address used can be beneficial for the following reasons:

1. **Maintaining privacy:** By rotating IP addresses, it becomes difficult for websites to trace multiple requests back to a single source. This helps keep your web scraping activities more confidential.

2. **Avoiding Blocks:** Websites often block IP addresses that send excessive scraping requests. By frequently changing your IP, you can distribute requests across many IPs and reduce the chances of getting blocked.

3. **Mitigating API Rate Limits:** APIs often limit how many requests can be made from one IP address. Rotating IPs allows you to spread requests over many IPs, helping stay under API rate limits and avoid 429 error (or "Too Many Requests" errors).

---

## Pre-requisites
Have a [Brightdata](https://brightdata.com/) account

### Step 1: Install libraries

In [5]:
%pip install requests
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Step 2: Set up proxy server

Assuming you've established your Bright Data account, the next step involves setting up your proxy server through this link: https://brightdata.com/cp/zones. In my experience, I opted for data center proxies due to their superior speed and cost-effectiveness.

I'll consistently choose data center proxies unless there are compelling reasons to reconsider. For example, when I still get blocked using data center proxies, I would choose to use residential proxies which is more expensive.

<img src="../assets/static/brightdata-create-datacenter-proxies-step1.png" width=500px alt="Step 1 - Create Datacenter proxies in Bright Data"><br>

Here I will choose to use premium data center proxies. Because this plan got a large pool of IP address I could rotate with:

For the other two plans, adding 1 more IP address would cost +$0.80/month...

<img src="../assets/static/brightdata-create-datacenter-proxies-step2.png" width=500px alt="Step 2 - Create Datacenter proxies in Bright Data"><br>


You can find the proxy pricing here (approximately $1.2/GB). Let's do a quick estimate: if each of your requests is around 50KB, sending 20,000 requests would only cost you $1.2!

<img src="../assets/static/brightdata-create-datacenter-proxies-step3.png" width=500px alt="Step 3 - Create Datacenter proxies in Bright Data">

### Step 3: IP rotating with Python library requests

In [6]:
import os
import random

import requests
from dotenv import load_dotenv

_ = load_dotenv()

url = "https://httpbin.co/ip"

proxy_hostname = os.getenv("PROXY_HOSTNAME")
proxy_username = os.getenv("PROXY_USERNAME")
proxy_password = os.getenv("PROXY_PASSWORD")

In [7]:
class ProxyServer:
    def __init__(self, proxy_username, proxy_password, proxy_hostname):
        self.proxy_username = proxy_username
        self.proxy_password = proxy_password
        self.proxy_hostname = proxy_hostname

    def __call__(self):
        rand_num = random.randint(1, 9999)
        # Reference: https://docs.brightdata.com/api-reference/proxy/rotate_ips
        ## Add -session parameter to this proxy to change IP address every requests is sent
        proxy = f"{self.proxy_username}-session-rand{rand_num}:{self.proxy_password}@{self.proxy_hostname}"
        return {
            "http": f"https://{proxy}",
            "https": f"https://{proxy}",
        }

proxy_servers = ProxyServer(proxy_username, proxy_password, proxy_hostname)
# call proxy_servers() to register a new IP proxy
# >> proxies = proxy_servers() 

In [8]:
# build a session object to preserve the session cookies (or login status, if applicable)
## throughout the process of sending web requests

session = requests.Session()
 
for idx in range(1, 11):
    response = session.request('GET', url=url, 
                proxies=proxy_servers()) # changing IP address every new requests
    # Note: you will see the web requests will be sent from different IP address 
    print("IP location metadata:", response.json())

IP location metadata: {'method': 'GET', 'ip': '94.139.59.101', 'country': 'US', 'timezone': 'America/Chicago'}
IP location metadata: {'method': 'GET', 'ip': '119.12.196.60', 'country': 'US', 'timezone': 'America/Chicago'}
IP location metadata: {'method': 'GET', 'ip': '161.129.163.147', 'country': 'US', 'timezone': 'America/Chicago'}
IP location metadata: {'method': 'GET', 'ip': '45.66.178.83', 'country': 'GB', 'region': 'England', 'city': 'London', 'timezone': 'Europe/London'}
IP location metadata: {'method': 'GET', 'ip': '89.32.132.58', 'country': 'DE', 'timezone': 'Europe/Berlin'}
IP location metadata: {'method': 'GET', 'ip': '93.180.229.23', 'country': 'US', 'timezone': 'America/Chicago'}
IP location metadata: {'method': 'GET', 'ip': '161.123.109.137', 'country': 'US', 'region': 'Texas', 'city': 'Austin', 'timezone': 'America/Chicago'}
IP location metadata: {'method': 'GET', 'ip': '161.123.79.31', 'country': 'US', 'region': 'New York', 'city': 'Newark', 'timezone': 'America/New_York

## Computing environment

In [9]:
%load_ext watermark

%watermark

# print out pypi packages used
%watermark --iversions

# date
%watermark -u -n -t -z

Last updated: 2024-03-17T07:44:11.463859+00:00

Python implementation: CPython
Python version       : 3.10.13
IPython version      : 8.22.2

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.75-060175-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit

requests: 2.31.0

Last updated: Sun Mar 17 2024 07:44:11UTC

